# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [ ]:
import sys
import os
# Add parent folder of current working directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
# Import module from parent folder

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [ ]:
from LLMHandler import LLMHandler

In [ ]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY_1")

In [ ]:
deals = ScrapedDeal.fetch(show_progress=True)

In [ ]:
len(deals)

In [ ]:
print(deals[44].describe())

In [ ]:
system_message = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price.

{{"deals": [
    {{
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    }},
    ...
]}}"""

In [ ]:
system_message = """You are a highly analytical AI that extracts the 5 best deals from a list. Your most critical task is to distinguish between a final price and a discount amount.

You will select deals based on two criteria:
1.  A detailed, high-quality product description.
2.  A clear, unambiguous, and final price.

CRUCIAL RULE ON PRICING: The price you extract MUST be the final, total cost of the item. It is never a discount amount.
-   If a deal mentions a price using words like "$XXX off", "save $XXX", "reduced by $XXX", or "a $XXX discount", you MUST IGNORE AND DISCARD that deal completely.
-   You must look for explicit pricing clues like "Price:", "Costs:", "For:", or a standalone number that clearly represents the total sale price.
-   If you have ANY doubt about whether a number is a final price or a discount, DO NOT include the deal. It is better to return fewer than 5 deals than to include one with an incorrect price.

For the 5 deals you select, respond strictly in JSON with no explanation. The `product_description` should be a summary of the item itself, not the deal's terms.

{{"deals": [
    {{
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    }},
    ...
]}}"""

In [ ]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [ ]:
deals

In [ ]:
print(user_prompt[:2000])

In [ ]:
handler = LLMHandler(system_message=system_message)
handler.TEXT_MODEL ="gemini-2.5-pro"

In [ ]:
from key_utils import get_next_key

In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

# 2. Setup parser
parser = PydanticOutputParser(pydantic_object=DealSelection)
api_key,_ = get_next_key()
# 3. Prompt
# The placeholder {user_input} is defined here
user_prompt_template = "{user_input}"
# The template is created from your messages
prompt = ChatPromptTemplate([
    ("system", system_message),
    ("user", user_prompt_template)
])

# 4. Model (ensure API key is configured)
# For example: from google.colab import userdata; api_key = userdata.get('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0, api_key=api_key)

# 5. Chain
chain = prompt | llm | parser

# 6. Call function
def get_recommendations(user_input: str):
    # This invoke call now works because the template only expects one variable: "user_input"
    return chain.invoke({"user_input": user_input})




In [ ]:
system_message

In [ ]:
# Example use

result = get_recommendations(user_prompt)




In [ ]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [ ]:
result

In [ ]:
result = get_recommendations()

In [ ]:
result.deals

In [ ]:
result.deals[1].price

In [ ]:
# Suppose `deals` is your list of Deal objects
deals_dict_list = [
    {
        "product_description": deal.product_description,
        "price": deal.price,
        "url": deal.url
    }
    for deal in result.deals
]

In [ ]:
deals_dict_list

In [ ]:
from agents.scanner_agent_langchain import ScannerAgent

In [ ]:
agent = ScannerAgent()
result = agent.scan()

In [ ]:
result